In [1]:
import joblib
import numpy as np
import pandas as pd


In [6]:
import joblib
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report

from sklearn.preprocessing import StandardScaler


from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier



from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV



In [3]:
ruta='datos/'
df_train_three=joblib.load(ruta+'df_train_datos2.pkl')
df_xtest_three=joblib.load(ruta+'df_xtest_datos2.pkl')



X=df_train_three.loc[:,~df_train_three.columns.isin(['target_duo', 'target_trio'])]
X_real_test=df_xtest_three.loc[:,~df_xtest_three.columns.isin(['target_duo', 'target_trio'])]
y2=df_train_three.loc[:,df_train_three.columns.isin(['target_duo'])]
y3=df_train_three.loc[:,df_train_three.columns.isin(['target_trio'])]


test_size=0.7
random_state=1000
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y2, \
    test_size=test_size, random_state=random_state)
X_train3, X_test3, y_train3, y_test3 = train_test_split(X, y3, \
    test_size=test_size, random_state=random_state)

In [9]:

log_reg = LogisticRegression(random_state=random_state)
nb = BernoulliNB()
knn = KNeighborsClassifier()
svm = SVC(random_state=random_state)
mlp = MLPClassifier(max_iter=500,random_state=random_state)
dt = DecisionTreeClassifier(random_state=random_state)
et = ExtraTreesClassifier(random_state=random_state)
rf = RandomForestClassifier(random_state=random_state)
xgb = XGBClassifier(random_state=random_state, verbosity=0)

clfs = [
        ('Logistic Regression', log_reg), 
        ('Naive Bayes', nb),
        ('K-Nearest Neighbors', knn), 
        ('SVM', svm), 
        ('MLP', mlp), 
        ('Decision Tree', dt), 
        ('Extra Trees', et), 
        ('Random Forest', rf), 
        ('XGBoost', xgb)
        ]


In [17]:
pipelines = []

scores_df = pd.DataFrame(columns=['Model', 'F1_Score', 'Precision', 'Recall', \
    'Accuracy'])


for clf_name, clf in clfs:
    print(f"trabajando clasificador {clf_name}")
    pipeline = Pipeline(steps=[
                                ('scaler', StandardScaler()),
                                ('classifier', clf)
                                ]
                        )
    pipeline.fit(X_train3, y_train3.values.ravel())

    print(f"fit realizado con clasificador {clf_name}")

    y_pred3 = pipeline.predict(X_test3)
    # F1-Score
    fscore = f1_score(y_test3, y_pred3, average='macro')
    # Precision
    pres = precision_score(y_test3, y_pred3, average='macro')
    # Recall
    rcall = recall_score(y_test3, y_pred3, average='macro')
    # Accuracy
    accu = accuracy_score(y_test3, y_pred3)

    pipelines.append(pipeline)

    scores_df = scores_df.append({
                                    'Model' : clf_name, 
                                    'F1_Score' : fscore,
                                    'Precision' : pres,
                                    'Recall' : rcall,
                                    'Accuracy' : accu,
                                    }, 
                                    ignore_index=True)

            


trabajando clasificador Naive Bayes
fit realizado con clasificador Naive Bayes
trabajando clasificador K-Nearest Neighbors
fit realizado con clasificador K-Nearest Neighbors
trabajando clasificador SVM
fit realizado con clasificador SVM
trabajando clasificador MLP
fit realizado con clasificador MLP
trabajando clasificador Decision Tree
fit realizado con clasificador Decision Tree
trabajando clasificador Extra Trees
fit realizado con clasificador Extra Trees
trabajando clasificador Random Forest
fit realizado con clasificador Random Forest
trabajando clasificador XGBoost
fit realizado con clasificador XGBoost


In [18]:
scores_df

,Model,F1_Score,Precision,Recall,Accuracy
0,Naive Bayes,0.477321,0.547113,0.487548,0.702189
1,K-Nearest Neighbors,0.584479,0.626050,0.569252,0.734560
2,SVM,0.521556,0.673769,0.515501,0.724050
3,MLP,0.561151,0.624019,0.548529,0.732492
4,Decision Tree,0.582584,0.581183,0.584202,0.701852
5,Extra Trees,0.634218,0.673080,0.615724,0.764382
6,Random Forest,0.626616,0.688646,0.605635,0.770803
7,XGBoost,0.598672,0.701746,0.578690,0.762169


In [30]:
pipelines[indice_mejor.values[0]]

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier', RandomForestClassifier(random_state=1000))])

In [31]:
# Preparamos salida.
indice_mejor=scores_df[['Accuracy']].idxmax()
print(f"mejor resultado en {scores_df['Model'][indice_mejor]}")
salida_prediccion=pipelines[indice_mejor.values[0]].predict(X_real_test)

mejor resultado en 6    Random Forest
Name: Model, dtype: object


In [32]:
df_xtest_three["respuesta"]=salida_prediccion
df_xtest_three["status_group"]=df_xtest_three["respuesta"].replace([0, 1,2],['functional', 'functional needs repair','non functional'])


In [96]:
# escribe respuesta
#df_xtest_three['status_group'].to_csv('datos/'+'respuesta2.csv') Su modelo tenía una puntuación de 0.77

In [33]:

from pathlib import Path
ruta='datos/'
nom='respuesta'
num=3
converted_num = "{}".format(num)
file=ruta+nom+converted_num+'.csv'
escrito=False
num=1
while escrito==False:
    converted_num = "{}".format(num)
    file=ruta+nom+converted_num+'.csv'
    path=Path(file)
    if path.is_file():
        num=num+1
        if num==100:
            escrito=False
            print("algo falla en el bucle")
    else:
        print(f"Aquí escribiría {file}")
        df_xtest_three['status_group'].to_csv(file) 
        escrito=True






Aquí escribiría datos/respuesta3.csv
